# Simulator


&emsp;The goal of our work is researching some approaches to solve the problem of maximizing the average number of pages read in multi-page publications. We want to learn how we need to lead the agent through the pages to cause her consuming as much content as possible. Unfortunately we have no special lab or resource to perform field experiments. That's why we need to build a good simulator that will mimic the agents behavior when she visits the pages we try to maximize the number of views.  
&emsp;The simulator receives from the outside directed graph that represents a structure of the content. Each vertex is a page and an edge from vertex A to vertex B express the link between page A and B, i.e. the next page that is presented to the agent after seeing page A may be page B.<br><br>
The agent behavior simulator is the core part of this project. The simulation is performed as following:<br>
    &emsp;**1)** At each round the agent is at some vertex of the graph - this is an abstraction for the agent looking at some page that we presented to her.<br>
    &emsp;**2)** On the same round, after watching the page, the agent chooses one of three actions: go to the next page, return to the previous page ( the one she saw at the previous round ), or stop watching the pages and leave.<br>
    &emsp;**3)** If the agent selected the "next page" action, the simulator choose the next page to present to the agent. <br>
    &emsp;**4)** The round finishes and the next round starts.<br><br>

&emsp;In this program we focus on second and the first step, the second step depends on the first one. We can think of it as if the agent liked the picture (or the sequence of pictures) or found it to be interesting and want to see the next one, or the opposite case, when she is bored with the current picture (or sequence of presented pictures) and don't want to keep watching.

    

### Simple version 

### Data structures and types

&emsp;The following data structures are simple abstract representations of the set of the pages and the links among them. Directed graph of the type Graph1 describes possible pages links. Vertexes of the graph of type Node1 represents one single page that the agent is supposed to see. If there is an edge from vertex A to vertex B ( B is in the set of A's neighbors) , page B may be presented to the agent after she saw page A and decided to proceed.<br>
<br>
&emsp;The last data structure type agent1 represents the agent, that traverse the pages of content. In our simulator she traverses the graph Graph1 until she leaves. The core part of out simulator is choiseFunction attribute of the agent. It is a function that defines the behavior of the agent with respect to each page (whether she leaves, proceeds or go back).

In [48]:
@enum UserAction  goBack goNext leave

struct Node1 
    id :: String
    NameInInput :: Any   
end


struct Graph1 
    graph :: Dict{Node1,Set{Node1}} 
end


struct User1 
   id :: String 
   ActionChoiseFunction  :: Any
end


### Auxiliary functions
Here we define simple auxiliary functions

In [49]:
# id_generator , get_node , printPath , getNeibhors

# Create a counter , used to produce unique ids
function id_generator()
    id = 0 
    return ()-> id = id + 1
end

nodeIdGenerator = id_generator()    # Nodes id generator
userIdGenerator = id_generator()    # Users id generator



# Return Node that corresponds to some name (that was given in input source , typicaly number)

function get_node(NameInInput, graph:: Graph1)
    nodes = graph.graph |> keys |> collect
    index = findfirst(node -> NameInInput == node.NameInInput,nodes) 
   return  nodes[index]
    end


# Given array of nodes , print their names devided by "=>"
# It is supposed to print the users path in the graph
function printPath(path :: Array{Node1,1})
    return   join(map(node-> node.NameInInput,path),"=>")
    
end

# Given node and graph , return node's neighbors
function getNeibhors(node :: Node1, graph :: Graph1) ::  Set{Node1}
        return graph.graph[node]
end;

### Producing the graph 

&emsp;The graph represents the structure of the content - possible order of the pages to be shown. Page B can be shown after page A only if there is edge (A,B) in the graph. It actually imposes constrains on the order the pages can be shown to the user. <br>

&emsp;In addition each node should contain unique identifiers or a callback function. It's required to simulate the reaction of the user to this particular page.<br>
&emsp;In other words, we need a way to separate the page from the others and define the attractiveness of it to the user. Some kind of API from the content side that will be used in deterministic or non-deterministic mechanism that simulates  agent's behavior after watching this page. 

The function below produces the simplest variant of such a graph of the type Graph1. It doesn't have any distinctions between the nodes, all the pages are considered the same. It provides only the content structure - the edges between the nodes. 

In [50]:
function createGraph(simple_tuples :: Array{Tuple{Int64,Int64},1})
    
    node_dict = Dict()
    graph = Graph1(Dict())
    
    create_new_node = (NameInInput) -> 
                        (()-> (id = string(nodeIdGenerator()); Node1(id,NameInInput)))
    
    for (n1,n2) in simple_tuples
        
        node1 = get!(create_new_node(n1),node_dict,n1)
        
        node2 = get!(create_new_node(n2),node_dict,n2)
        
        neighbors = get!(Set,graph.graph,node1)
        
        push!(neighbors,node2)      
        
    end

  return graph  
    
end;

### Main interface functions

&emsp;The following functions are actually the most important parts of the simulator. The first one defines user behavior : which action will be chosen by the agent after watching some particular page (visiting some particular node ). <br>
The second function defines which new page will be presented to the agent if she decided to proceed. It will connect the simulator to some external algorithm that will attempt to maximize the number of pages visited.



&emsp;The following implementations are trivial : 1) In order to choose the next action the inner function of the agent is called 2)The next page to show to the user is selected uniformly among neighbors of the current page in the graph. 

In [51]:



function chooseNextAction(user :: User1, node :: Node1) :: UserAction
    user.ActionChoiseFunction(node)
end



function getNextNode(graph :: Graph1, path :: Array{Node1,1})
    currentNode = path[end]
    return rand(getNeibhors(currentNode,graph))
    end;


## Main function 

&emsp;Here the simulator is ran for one agent. Given initial node and content graph, the following function simulates agent's traversal in the set of pages. It assumes that the agent can make 3 actions: 1) Press "next" -- proceed to the next page. 2) Press "back" - go back to the previous page that she visited 3) Leave the pages -- it is always the last action in the simulation for the current agent.<br>
In response to each action the current state of the simulator is updated , and the information about user's traversals is collected. <br>
There are two important records here : path and current node . <br> <br>
&emsp;1) **path** -- list of unique nodes that were visited by the agent sorted by the time (round index) they were revealed.<br>
Walking back and forth is allowed only along the path, and it happens when the agent presses next and back. When the agent is at the last node ( path\[end\] ) and press "next", she traverses to the new node that is returned by getNextNode function. <br> 
&emsp;2) **current node** -- the node the agent is currently at i.e the page the user watches now. <br><br>
&emsp;For example: at the beginning the path contains only the initial node N1 and current node is also N1. Than the user presses "next" and the new node N2 is provided by getNextNode  , the path is (N1,N2) and current node is N2. Than the user presses "back" , the path remains the same and current node turns equal to N1 . Than the user presses "next" and the state turns to be path =(N1,N2) current node = N2 .....

In [52]:
function performWalk(user :: User1, graph :: Graph1, initialNode :: Node1)
    
    currentNode = initialNode
    path = Node1[currentNode]
    nodesVisited = Node1[currentNode]
    currentIndex = 1
    stepCounter = 0

    while true
              
       nextAction = chooseNextAction(user, currentNode)
        
       # The agent is somewhere on previous pages that he visited and press next
        if nextAction == goNext && currentIndex < length(path)
            currentIndex += 1
            currentNode = path[currentIndex]
            stepCounter += 1
            push!(nodesVisited,currentNode)
        # The agent is somewhere on previous pages that he visited and press back
        elseif nextAction == goBack && currentIndex > 1
            currentIndex -= 1
            currentNode = path[currentIndex]
            stepCounter += 1
            push!(nodesVisited,currentNode)
        # The agent is on the last page that he visited and press next
        elseif nextAction == goNext && currentIndex == length(path)
            currentIndex += 1
            currentNode = getNextNode(graph,path)
            stepCounter += 1
            push!(path,currentNode)
            push!(nodesVisited,currentNode)
        # The agent press leave
        elseif nextAction == leave
            break
        # Unexpected combination of conditions
        else 
            stepCounter += 1

            push!(nodesVisited,currentNode)
            # Throw exception
            # throw(ErrorException("Something went wrong path $path , current index $currentIndex , step counter $stepCounter, action $nextAction"))
       end
        
    end

    return Dict("path" => path, "nodesVisited" => nodesVisited)
    end;


### The example of run 

The user has a simple policy , choose next node randomly.
Initial node is node 1 .


In [144]:
user = User1("1",x -> rand(instances(UserAction)))

graph = createGraph([(1,2),(2,3),(1,3),(3,4)])

initial_node = get_node(1,graph)

result = performWalk(user,graph,initial_node)

printPath(result["nodesVisited"])

LoadError: [91mKeyError: key Node1("368", 4) not found[39m